# Chapter 5. 순환 신경망(RNN)

지금까지 살펴본 신경망은 `피드포워드(feed forward)` 라는 유형의 신경망  
- `피드 포워드` : 흐름이 단방향인 신경망
  - 장점 : 구성이 단순하여 구조를 이해하기 쉽고 많은 문제에 응용 가능
  - 단점 : 시계열 데이터를 잘 다루지 못함

#### 이번 단원의 목표 : `피드포워드`의 문제점을 지적하고, `RNN`의 구조를 이해해 그 문제점을 해결

## 5.1 확률과 언어 모델

### 5.1.1 `word2vec`을 확률 관점에서 바라보다

#### `word2vec` 의 `CBOW` 모델

$ w_1, w_2, ..., w_{T} $ 라는 단어열로 표현되는 말뭉치을 생각

$w_{t-1}$과 $w_{t+1}$이 주어졌을 때 타깃이 $w_{t}$가 될 확률 (윈도우 크기가 1일 때)

$$ P(w_t | w_{t-1}, w_{t+1})$$

맥락을 좌우 대칭이 아니라 왼쪽 윈도우만으로 한정해서 생각해보자.  
<img src="./master/images/fig 5-2.png" width=600/>

$$ P(w_t | w_{t-2}, w_{t-1})$$

`교차 엔트로피 오차`를 이용해서 구한 위 식의 손실 함수  
$$L = -\log P(w_t | w_{t-1}, w_{t+1})$$

### 5.1.2 언어 모델

- `언어 모델` : 단어 나열에 확률을 부여. 특정한 단어의 시퀀스에 대해서, 그 시퀀스가 얼마나 자연스러운지 확률로 평가
- `언어 모델`은 새로운 문장을 생성하는 용도로도 이용할 수 있다.
- `CBOW` 모델은 단어 순서를 무시하기 때문에 `언어 모델`로 적합하지 않다.
- `RNN(순환신경망)`은 긴 시계열 데이터에도 대응할 수 있다.

## 5.2 RNN이란

### 5.2.1 순환하는 신경망

순환하기 위해서는 `닫힌 경로`가 필요

<p align = 'left'><img src="./master/images/fig 5-6.png" height=250 /></p>  
  
- $x_{t}$ : 각 단어의 분산 표현 (단어 벡터)

### 5.2.2 순환 구조 펼치기

<p align = 'left'><img src="./master/images/fig 5-8.png" height=250 /></p>  


<p align = 'left'><img src="./master/images/e 5-9.png" /></p>  

### 5.2.3 BPTT

<p align = 'left'><img src="./master/images/fig 5-10.png" height=200 /></p>  


`RNN`에서의 오차역전파법은 '시간 방향으로 펼친 신경망의 오차역전파법'이란 뜻으로 `BPTT(Backpropagation Through Time)`라고 한다.

시계열 데이터의 시간 크기가 커지는 것에 비례하여 `BPTT`가 소비하는 컴퓨팅 자원도 증가한다는 단점이 있다.

### 5.2.4 Truncated BPTT

`BPTT`의 문제를 해결하기 위해, 신경망을 적당한 지점에서 잘라서 작은 신경망 여러 개로 만드는 `Truncated BPTT`를 사용한다.

<p align = 'left'><img src="./master/images/fig 5-14.png" height=300 /></p>  


### 5.2.5 Truncated BPTT의 미니배치 학습

<p align = 'left'><img src="./master/images/fig 5-15.png" height=500 /></p>  


- 길이가 1000인 시계열 데이터
- 시각의 길이를 10개로 자르는 `Truncated BPTT` 로 학습
- 미니배치의 수 : 2개

## 5.3 RNN 구현

<p align = 'left'><img src="./master/images/fig 5-17.png" height=300 /></p>  


- `Time RNN` 계층 내에서 한 단계의 작업을 수행하는 계층 : `RNN` 계층
- $T$ 개 단계분의 작업을 한꺼번에 처리하는 계층 : `Time RNN` 계층

### 5.3.1 RNN 계층 구현

<p align = 'left'><img src="./master/images/fig 5-18.png" height=200 /></p>  
  
#### forward 구현


In [2]:
import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next

<p align = 'left'><img src="./master/images/fig 5-20.png" height=300 /></p>  
  
#### backward 구현

In [3]:

import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.t)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

### 5.3.2 Time RNN 계층 구현

In [7]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False): # stateful : 은닉 상태를 인계받을지
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
        self.h, self.dh = None
        self.stateful = stateful
        
    def set_state(self, h):
        self.h = h
        
    def reset_state(self):
        self.h = None
        
#forward()#####################################################################################
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty((N, T, H), dtype='f')
        
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
            
        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)
            
        return hs

#backward()######################################################################################
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 합산된 기울기 (?)
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
                
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        
        return dxs